<a href="https://colab.research.google.com/github/ev1025/happy_dog_map/blob/main/%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 강아지 행복지도

## **로컬DB happydog에서 크롤링파일 가져오기**

In [ ]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
import math

import pymysql
import pymysql.cursors

In [ ]:
# mysql 연결자 생성
conn = pymysql.connect(host='127.0.0.1',   # db주소
                       user='root',        # db유저아이디
                       password='s10e534', # db비밀번호
                       db='happydog', # db이름
                       charset='utf8',     # 한글설정
                       cursorclass = pymysql.cursors.DictCursor)


cur = conn.cursor() # 커서 생성

In [ ]:
# 일반 크롤링, 강아지도 사이트 크롤링 데이터 합치기
sql = '''
select * 
from star_dog_map
union
select * 
from happy_dog_list
'''

In [ ]:
# 데이터 가져와서 map_list에 저장
cur.execute(sql)
map_list = cur.fetchall()
cur.close()
print(map_list[0])

{'상호명': '조용한저녁', '업종': '양식', '리뷰 수': 0, '별점': '4.83', '행정구': '마포구', '전화번호': '0507-1431-4540'}


## 데이터 전처리

In [ ]:
df = pd.DataFrame(map_list)

**중복확인**
- 크롤링 시기에 따른 중복 발생

In [ ]:
drop_index =df[(df['상호명'].duplicated())&(df['전화번호'].duplicated())].index
df.drop(drop_index,inplace=True)

In [ ]:
df[(df['상호명'].duplicated())&(df['전화번호'].duplicated())]

,상호명,업종,리뷰 수,별점,행정구,전화번호


**업종명이 불명확한 부분이 많아서 업종명을 정리**

In [ ]:
df['업종'].unique()

array(['양식', '카페,디저트', '햄버거', '국밥', '카페', '도시락,컵밥', '한식', '바(BAR)', '죽',
       '다이어트,샐러드', '뜨개', '커피번', '아이스크림', '빙수', '베이글', '베이커리',
       '스파게티,파스타전문', '호두과자', '과일,주스전문점', '꽃집,꽃배달', '라이브카페', '도넛', '서점',
       '와인', '테이크아웃커피', '복합문화공간', '퓨전음식', '피자', '애견카페', '브런치', '샌드위치',
       '플라워카페', '맥주,호프', '태국음식', '케이크전문', '다방', '육류,고기요리', '떡볶이', '아시아음식',
       '스페인음식', '멕시코,남미음식', '주류', '이탈리아음식', '돼지고기구이', '일식당', '요리주점',
       '사진,스튜디오', '카레', '채식,샐러드뷔페', '테마카페', '반려동물미용', '해물,생선요리', '스테이크,립',
       '72420', '양갈비', '곱창,막창,양', '만두', '덮밥', '생선회', '프랑스음식', '돈가스',
       '포장마차', '일식튀김,꼬치', '닭갈비', '북카페', '종합분식', '찌개,전골', '이자카야', '인도음식',
       '미용실', '스포츠시설', '네일아트,네일샵', '반려견놀이터', '애견용품', '족발,보쌈', '조개요리',
       '치킨,닭강정', '주꾸미요리', '분식', '닭발', '베트남음식', '닭볶음탕', '소고기구이', '냉면',
       '백숙,삼계탕', '장어,먹장어요리', '갤러리카페', '차', '전통,민속주점', '우동,소바', '순대,순댓국',
       '중식당', '일본식라면', '술집', '오뎅,꼬치', '커피가공,제조', '장소대여', '스크린골프장',
       '반려동물용품', '반려동물', '반려동물호텔', '애견훈련', '약국', '동물병원', '애견의류', '동물약국',
       '

In [ ]:
# 큰 카테고리로 분류
# 동반식당
df['업종'].replace(['양식','햄버거','국밥','도시락,컵밥','한식','일본식라면','바(BAR)','퓨전음식','죽','다이어트,샐러드','뜨개','와인','스파게티,파스타전문'
                 ,'스테이크,립','피자','브런치','샌드위치','맥주,호프','태국음식','이탈리아음식','육류,고기요리','떡볶이','아시아음식','스페인음식','멕시코,남미음식'
                 ,'주류','중식당','덮밥','요리주점','프랑스음식','일식당','돼지고기구이', '술집', '이자카야', '카레', '찌개,전골', '치킨,닭강정', '주꾸미요리', '분식', 
                  '닭발', '베트남음식', '닭볶음탕', '생선회', '곱창,막창,양', '소고기구이', '냉면', '백숙,삼계탕','전통,민속주점',  '포장마차', '게요리', '우동,소바', '토스트', '순대,순댓국'
                  , '오뎅,꼬치',  '닭갈비', '채식,샐러드뷔페', '해물,생선요리', '72420', '양갈비', '만두', '일식튀김,꼬치', '종합분식', '인도음식', '족발,보쌈', '조개요리','장어,먹장어요리'
                  , '돈가스'],'식당(동반)',inplace=True)
# 동반카페
df['업종'].replace(['카페,디저트','카페','커피번','아이스크림','빙수','베이글','베이커리','호두과자','과일,주스전문점','꽃집,꽃배달','라이브카페','도넛','북카페',
                  '테이크아웃커피','플라워카페','케이크전문','다방','갤러리카페', '차', '보드카페','예식장','커피가공,제조', '사주카페', '테마카페'],'카페(동반)', inplace=True)

# 애견미용
df['업종'].replace('반려동물미용','애견미용', inplace=True)

# # 애견위탁
df['업종'].replace(['반려동물호텔','애견훈련'],'애견위탁', inplace=True)

# 애견용품
df['업종'].replace(['반려동물분양','반려동물용품', '애견사료','애완동물', '애견의류'],'애견용품', inplace=True)

# 동물약국
df['업종'].replace('약국','동물약국', inplace=True)

# 간식, 기타 취미생활
df['업종'].replace(['반려견놀이터','제조업','동물사료제조','뜨개', '아기사진전문', '네일아트,네일샵', '셀프,대여스튜디오', '서점','복합문화공간','장소대여','사진,스튜디오',
                  '공방', '스크린골프장','스포츠시설','애견카페'],'취미생활', inplace=True)

In [ ]:
df['업종'].unique()

array(['식당(동반)', '카페(동반)', '취미생활', '애견미용', '미용실', '애견용품', '반려동물', '애견위탁',
       '동물약국', '동물병원', '치과', '교육원,교육센터', '종합도소매', '장식용제품제조', '기업', '세무사',
       '건강,의료', '수족관', '문구,팬시용품', '가정,생활용품', '인테리어디자인', '가공식품', '인테리어소품',
       '패션', '식료품제조', '인터넷상거래', '직업,기술학원', '광고대행', '창업,소호', '정육점',
       '종합생활용품', '의류제작'], dtype=object)

In [ ]:
# 업종이 반려동물인 경우
df[df['업종']=='반려동물']

,상호명,업종,리뷰 수,별점,행정구,전화번호
1316,하울팟케어클럽 한남점,반려동물,186,nan,용산구,02-6223-3480
1326,엘리티레아,반려동물,0,nan,용산구,nan
1369,쿠이다르,반려동물,3,nan,용산구,0507-1390-7005
1693,빵댕잇,반려동물,14,nan,종로구,0507-1391-4670
1944,프랑소와펫 서래점,반려동물,56,4.78,서초구,0507-1472-3364
1947,프랑소와펫 뱅뱅사거리점,반려동물,13,nan,서초구,0507-1467-3406
1949,위드랜드 강남점,반려동물,27,4.5,서초구,02-512-2176
1950,하울팟 케어클럽 서초본점,반려동물,109,nan,서초구,0507-1487-1108
2054,유어마이독,반려동물,6,nan,서초구,0507-1444-5025
2155,프랑소와펫 신사점,반려동물,39,4.89,강남구,02-547-3362


In [ ]:
# 업종이 반려동물인 경우 세분화해서 넣어줌
df.loc[[3580,],'업종'] = '동물약국'
df.loc[[3336,],'업종'] = '애견미용'
df.loc[[1693,2529,2799,2804,2899,3562,3741,3742,3743,3881],'업종'] = '취미생활'
df.loc[[1944,1947,2164,3268,3335,],'업종'] = '애견용품'
df.loc[[1316,1326,1369,1949,1950,2054,2155,2156,2157,2162,2163,2165,2166,2184,2187,
        2354,2355,2460,2479,2966,3147,3333,3334,3444,3445,3540,3665,3746,3843,3847],'업종'] = '애견위탁'

In [ ]:
df[df['업종']=='반려동물']

,상호명,업종,리뷰 수,별점,행정구,전화번호


In [ ]:
df['업종'].unique()

array(['식당(동반)', '카페(동반)', '취미생활', '애견미용', '미용실', '애견용품', '애견위탁', '동물약국',
       '동물병원', '치과', '교육원,교육센터', '종합도소매', '장식용제품제조', '기업', '세무사', '건강,의료',
       '수족관', '문구,팬시용품', '가정,생활용품', '인테리어디자인', '가공식품', '인테리어소품', '패션',
       '식료품제조', '인터넷상거래', '직업,기술학원', '광고대행', '창업,소호', '정육점', '종합생활용품',
       '의류제작'], dtype=object)

In [ ]:
# # 이상치 확인용
# df[df['업종']=='의류제작']

In [ ]:
df.loc[[1702,3849,3493],'업종'] = '애견용품'
df.loc[[2053,3190,3191,3496],'업종'] = '취미생활'
df.loc[[1659],'업종'] = '애견위탁'
df.loc[[1803],'업종'] = '동물병원'
df.drop([1916],axis=0,inplace=True)

In [ ]:
# 완전히 잘못 수집된 정보 제거
del_list = ['직업,기술학원','미용실','인테리어소품','편입학원','치과','장식용제품제조','세무사','건강,의료','수족관',
            '문구,팬시용품', '가정,생활용품', '인테리어디자인','가공식품','패션','광고대행', '창업,소호', '정육점']

del_list = '|'.join(del_list)
del_index = df[df['업종'].str.contains(del_list)].index

df.drop(index=del_index, inplace=True)

In [ ]:
df['업종'].unique()

array(['식당(동반)', '카페(동반)', '취미생활', '애견미용', '애견용품', '애견위탁', '동물약국', '동물병원'],
      dtype=object)

**별점이 없는 장소는 평균별점을 넣어준다.**

In [ ]:
# 별점이 없는(nan) 행들 0으로 변경, 타입을 float으로 변경
nan_index = df[df['별점'].str.contains('nan')]['별점'].index
df.loc[nan_index,'별점'] = df.loc[nan_index,'별점'].str.replace('nan', '0')
df['별점'] = df['별점'].astype(float)

In [ ]:
# 별점이 nan이 아닌 값의 평균을 구함
not_nan_index = [i for i in df.index.tolist() if i not in nan_index.tolist()]
star_nnul = df.loc[not_nan_index]
star_nnul = round(star_nnul.groupby('업종')[['별점']].mean(),2).to_dict()
star_dict = star_nnul['별점']
star_dict

{'동물병원': 4.51,
 '동물약국': 4.4,
 '식당(동반)': 4.57,
 '애견미용': 4.61,
 '애견용품': 4.48,
 '애견위탁': 4.63,
 '취미생활': 4.61,
 '카페(동반)': 4.58}

In [ ]:
# 해당 업종의 평균값으로 0인 값을 채워줌
for key, value in star_dict.items():
    rep_index = df[(df['업종']=='key')|df['별점']==0].index
    df.loc[rep_index,'별점'] = value

# 0인 가게 확인
df[df['별점']==0]

,상호명,업종,리뷰 수,별점,행정구,전화번호


## 지표 만들기

**병원을 0.225, 기타 반려견을 위한 시설은 0.125, 단순동반시설은 0.1점 부여**

In [ ]:
df['업종'].value_counts()

업종
카페(동반)    907
애견용품      645
동물병원      643
애견미용      589
동물약국      483
식당(동반)    324
애견위탁      154
취미생활      120
Name: count, dtype: int64

In [ ]:
# 업종별 지표
cat_name = df['업종'].unique().tolist()
cat_name
weight_indicators = pd.DataFrame({'업종' : cat_name, '지표점수':[0.1, 0.1, 0.1, 0.125, 0.125, 0.125, 0.1, 0.225]})
weight_indicators

,업종,지표점수
0,식당(동반),0.100
1,카페(동반),0.100
2,취미생활,0.100
3,애견미용,0.125
4,애견용품,0.125
5,애견위탁,0.125
6,동물약국,0.100
7,동물병원,0.225


**25개 구의 반려동물 수 수집**
- 공공데이터 포털을 이용
- 한 번에 통합된 자료가 없어서 각 구별로 확인하여 추가 (데이터마다 시기의 차이가 있음)
- 강아지와 고양이를 분류한 자료가 있고, 분류하지 않은 자료가 있었음


In [ ]:
s_pop = pd.DataFrame({'행정구':['관악구','영등포구','마포구','용산구','서대문구','강남구','종로구','성북구','중구',
                                '송파구','강동구','동작구','도봉구','노원구','강북구','중랑구','동대문구',
                                '성동구','광진구','양천구','구로구','강서구','금천구','은평구','서초구'],
                      '동물 수' :[25027,14757,24831,17717,16944,31371,9753,21877,8442,30488,18281,18989,14676,
                                 28129,14986,24810,15811,16943,16280,24563,8920,23786,13538,28578,23953]})
s_pop

,행정구,동물 수
0,관악구,25027
1,영등포구,14757
2,마포구,24831
3,용산구,17717
4,서대문구,16944
5,강남구,31371
6,종로구,9753
7,성북구,21877
8,중구,8442
9,송파구,30488


In [ ]:
# 구별 반려동물 수 추가
all_data = pd.merge(df, s_pop, how='left',on='행정구')
all_data

,상호명,업종,리뷰 수,별점,행정구,전화번호,동물 수
0,조용한저녁,식당(동반),0,4.83,마포구,0507-1431-4540,24831
1,카페 스핀오프,카페(동반),362,4.58,마포구,0507-1351-1078,24831
2,A SLICE,카페(동반),238,4.67,마포구,0507-1307-2724,24831
3,니즈버거 신촌점,식당(동반),402,4.66,마포구,02-336-0249,24831
4,리카르도,카페(동반),320,4.52,마포구,070-4827-5701,24831
...,...,...,...,...,...,...,...
3860,맑은수약국,동물약국,314,4.51,영등포구,0507-1358-2008,14757
3861,라이프동물병원,동물병원,234,4.57,영등포구,02-832-7975,14757
3862,경인동물의료센터,동물병원,329,4.57,영등포구,02-571-8275,14757
3863,개빵집,취미생활,11,4.51,영등포구,0507-1362-8728,14757


In [ ]:
# 구별 업종별 개수
df_cat_num = df.groupby(['행정구','업종'],as_index=False)['전화번호'].count()
df_cat_num.rename(columns={'전화번호':'개수'},inplace=True)
df_cat_num

,행정구,업종,개수
0,강남구,동물병원,88
1,강남구,동물약국,63
2,강남구,식당(동반),47
3,강남구,애견미용,60
4,강남구,애견용품,63
...,...,...,...
194,중랑구,애견미용,9
195,중랑구,애견용품,16
196,중랑구,애견위탁,6
197,중랑구,취미생활,6


In [ ]:
# 구별 업종별 개수 합치기
all_data = pd.merge(all_data, df_cat_num,how='left', on=['행정구','업종'])
all_data

,상호명,업종,리뷰 수,별점,행정구,전화번호,동물 수,개수
0,조용한저녁,식당(동반),0,4.83,마포구,0507-1431-4540,24831,53
1,카페 스핀오프,카페(동반),362,4.58,마포구,0507-1351-1078,24831,70
2,A SLICE,카페(동반),238,4.67,마포구,0507-1307-2724,24831,70
3,니즈버거 신촌점,식당(동반),402,4.66,마포구,02-336-0249,24831,53
4,리카르도,카페(동반),320,4.52,마포구,070-4827-5701,24831,70
...,...,...,...,...,...,...,...,...
3860,맑은수약국,동물약국,314,4.51,영등포구,0507-1358-2008,14757,8
3861,라이프동물병원,동물병원,234,4.57,영등포구,02-832-7975,14757,7
3862,경인동물의료센터,동물병원,329,4.57,영등포구,02-571-8275,14757,7
3863,개빵집,취미생활,11,4.51,영등포구,0507-1362-8728,14757,3


In [ ]:
# 구별 업종별 할당되는 강아지 수
all_data['할당량'] = round(all_data['동물 수'] / all_data['개수']).astype(int)
all_data

,상호명,업종,리뷰 수,별점,행정구,전화번호,동물 수,개수,할당량
0,조용한저녁,식당(동반),0,4.83,마포구,0507-1431-4540,24831,53,469
1,카페 스핀오프,카페(동반),362,4.58,마포구,0507-1351-1078,24831,70,355
2,A SLICE,카페(동반),238,4.67,마포구,0507-1307-2724,24831,70,355
3,니즈버거 신촌점,식당(동반),402,4.66,마포구,02-336-0249,24831,53,469
4,리카르도,카페(동반),320,4.52,마포구,070-4827-5701,24831,70,355
...,...,...,...,...,...,...,...,...,...
3860,맑은수약국,동물약국,314,4.51,영등포구,0507-1358-2008,14757,8,1845
3861,라이프동물병원,동물병원,234,4.57,영등포구,02-832-7975,14757,7,2108
3862,경인동물의료센터,동물병원,329,4.57,영등포구,02-571-8275,14757,7,2108
3863,개빵집,취미생활,11,4.51,영등포구,0507-1362-8728,14757,3,4919


In [ ]:
# 반려동물 할당량이 적은 순서로 순위를 냄
df_score = all_data.groupby(['업종','행정구'],as_index=False)[['할당량']].mean()
df_score['개별순위'] = df_score.groupby(['업종'])['할당량'].rank(method='min',ascending=True).astype(int)
df_score = pd.merge(df_score, weight_indicators, how='left', on=['업종'])
df_score

,업종,행정구,할당량,개별순위,지표점수
0,동물병원,강남구,356.0,2,0.225
1,동물병원,강동구,345.0,1,0.225
2,동물병원,강북구,1499.0,15,0.225
3,동물병원,강서구,3964.0,25,0.225
4,동물병원,관악구,3575.0,24,0.225
...,...,...,...,...,...
194,카페(동반),용산구,201.0,2,0.100
195,카페(동반),은평구,1191.0,20,0.100
196,카페(동반),종로구,191.0,1,0.100
197,카페(동반),중구,222.0,3,0.100


In [ ]:
# 낮을수록 점수가 높도록 점수 계산용 컬럼 생성
df_score['역순위'] = df_score.groupby(['업종'])['할당량'].rank(method='average',ascending=False)

# 순위별로 가중치를 계산 지표점수 * (순위점수 / 행정구 수)
df_score['개별점수'] = round((df_score['역순위']/25)*df_score['지표점수'],3)
df_score

,업종,행정구,할당량,개별순위,지표점수,역순위,개별점수
0,동물병원,강남구,356.0,2,0.225,24.0,0.216
1,동물병원,강동구,345.0,1,0.225,25.0,0.225
2,동물병원,강북구,1499.0,15,0.225,11.0,0.099
3,동물병원,강서구,3964.0,25,0.225,1.0,0.009
4,동물병원,관악구,3575.0,24,0.225,2.0,0.018
...,...,...,...,...,...,...,...
194,카페(동반),용산구,201.0,2,0.100,24.0,0.096
195,카페(동반),은평구,1191.0,20,0.100,6.0,0.024
196,카페(동반),종로구,191.0,1,0.100,25.0,0.100
197,카페(동반),중구,222.0,3,0.100,23.0,0.092


In [ ]:
# 전체 점수 확인결과 너무 점수가 낮게 나옴
# 시설이 없는 행정구의 경우 지표가 너무 낮게 나옴
a =[]
for i in df['행정구'].unique():
    print(f'{i} :{sum(df_score[df_score["행정구"]==i].loc[:,"개별점수"])}')

마포구 :0.7919999999999999
용산구 :0.6250000000000001
서대문구 :0.51
은평구 :0.42900000000000005
종로구 :0.601
성북구 :0.6160000000000001
중구 :0.7669999999999999
서초구 :0.7250000000000001
강남구 :0.9199999999999999
송파구 :0.615
강동구 :0.7929999999999999
동작구 :0.38
도봉구 :0.56
노원구 :0.42200000000000004
강북구 :0.4989999999999999
중랑구 :0.292
동대문구 :0.428
성동구 :0.44599999999999995
광진구 :0.42100000000000004
구로구 :0.397
양천구 :0.49500000000000005
강서구 :0.28900000000000003
영등포구 :0.298
관악구 :0.3340000000000001
금천구 :0.246


In [ ]:
# 점수를 다시 만들기 위해 컬럼 삭제
df_score.drop(['개별점수'],axis=1, inplace=True)

In [ ]:
# 개별점수 부여 (업종별,행정구별 점수)
for i in df_score['업종'].unique().tolist():
    cat_num = len(df_score[df_score['업종']==i])     # 업종별 지역 개수
    cat_index = df_score[df_score['업종']==i].index  # 업종별 인덱스

    #  # 개별점수 컬럼을 새로 만들어서 넣어줌
    df_score.loc[cat_index,'개별점수'] =  \
        round((df_score.loc[cat_index,'역순위']/cat_num) * df_score.loc[cat_index,'지표점수'],3) # (순위계산용 점수 / 전체 행정구 수) * 업종별 지표점수 

# 만들어진 점수를 정수화
df_score['개별점수'] = round((df_score['개별점수']/df_score['지표점수'])*100).astype(int) 
df_score

,업종,행정구,할당량,개별순위,지표점수,역순위,개별점수
0,동물병원,강남구,356.0,2,0.225,24.0,96
1,동물병원,강동구,345.0,1,0.225,25.0,100
2,동물병원,강북구,1499.0,15,0.225,11.0,44
3,동물병원,강서구,3964.0,25,0.225,1.0,4
4,동물병원,관악구,3575.0,24,0.225,2.0,8
...,...,...,...,...,...,...,...
194,카페(동반),용산구,201.0,2,0.100,24.0,96
195,카페(동반),은평구,1191.0,20,0.100,6.0,24
196,카페(동반),종로구,191.0,1,0.100,25.0,100
197,카페(동반),중구,222.0,3,0.100,23.0,92


In [ ]:
# 구별 총점 및 순위
temp = {}
for i in df['행정구'].unique():
    temp[i] =int(sum(df_score[df_score["행정구"]==i].loc[:,"개별점수"])/8)

gu_score = pd.DataFrame.from_dict(temp, orient='index', columns=['행정구점수'])
gu_score.reset_index(drop=False, inplace=True)
gu_score.rename(columns={'index':'행정구'},inplace=True)
gu_score['행정구순위'] = gu_score['행정구점수'].rank(ascending=False).astype(int)
gu_score.head()

,행정구,행정구점수,행정구순위
0,마포구,80,2
1,용산구,61,8
2,서대문구,49,12
3,은평구,39,18
4,종로구,63,7


In [ ]:
# 테마별 점수와 순위 선정
# 건강점수
health_score = df_score.query('업종==("동물병원","동물약국")').groupby(['행정구'],as_index=False)[['개별점수']].sum()
health_score['개별점수'] = round(health_score['개별점수']/2)
health_score.insert(0,'테마','건강점수')
health_score['테마순위'] = health_score['개별점수'].rank(ascending=False).astype(int)

# 취미점수
hobby_score = df_score.query("업종==('식당(동반)', '카페(동반)', '취미생활')").groupby('행정구',as_index=False)[['개별점수']].sum()
hobby_score['개별점수'] = round(hobby_score['개별점수']/3)
hobby_score.insert(0,'테마','취미점수')
hobby_score['테마순위'] = hobby_score['개별점수'].rank(ascending=False).astype(int)

# 관리점수
care_score = df_score.query("업종==('애견미용', '애견용품', '애견위탁')").groupby('행정구',as_index=False)[['개별점수']].sum()
care_score['개별점수'] = round(care_score['개별점수']/3)
care_score.insert(0,'테마','관리점수')
care_score['테마순위'] = care_score['개별점수'].rank(ascending=False).astype(int)

tem_score =  pd.concat([health_score,hobby_score,care_score])
tem_score['개별점수'] = tem_score['개별점수'].astype(int)

In [ ]:
# 테마점수 + 전체 점수
tem_score = pd.merge(tem_score, gu_score, how='left',on =['행정구'])
tem_score.rename(columns={'개별점수':'테마점수'},inplace=True)
tem_score

,테마,행정구,테마점수,테마순위,행정구점수,행정구순위
0,건강점수,강남구,96,2,92,1
1,건강점수,강동구,100,1,75,3
2,건강점수,강북구,60,9,50,11
3,건강점수,강서구,4,25,32,23
4,건강점수,관악구,8,24,36,21
...,...,...,...,...,...,...
70,관리점수,용산구,63,7,61,8
71,관리점수,은평구,49,13,39,18
72,관리점수,종로구,37,19,63,7
73,관리점수,중구,80,3,74,4


In [ ]:
# 구별 업종 점수
cat_score = df_score.pivot(index='행정구',columns='업종',values='개별점수').reset_index()
cat_score

업종,행정구,동물병원,동물약국,식당(동반),애견미용,애견용품,애견위탁,취미생활,카페(동반)
0,강남구,96.0,96.0,96.0,88.0,84.0,100.0,100.0,76.0
1,강동구,100.0,100.0,46.0,92.0,92.0,96.0,4.0,70.0
2,강북구,44.0,76.0,29.0,96.0,40.0,24.0,44.0,52.0
3,강서구,4.0,4.0,21.0,12.0,20.0,64.0,92.0,44.0
4,관악구,8.0,8.0,38.0,4.0,56.0,68.0,96.0,16.0
5,광진구,20.0,24.0,67.0,32.0,8.0,80.0,68.0,70.0
6,구로구,28.0,48.0,33.0,40.0,96.0,48.0,12.0,12.0
7,금천구,32.0,16.0,4.0,24.0,32.0,16.0,60.0,4.0
8,노원구,60.0,60.0,12.0,56.0,52.0,32.0,8.0,32.0
9,도봉구,68.0,72.0,25.0,100.0,76.0,28.0,28.0,28.0


In [ ]:
# 태블로용 점수 데이터
all_score = pd.merge(tem_score ,cat_score, how='left', on='행정구')
all_score

,테마,행정구,테마점수,테마순위,행정구점수,행정구순위,동물병원,동물약국,식당(동반),애견미용,애견용품,애견위탁,취미생활,카페(동반)
0,건강점수,강남구,96,2,92,1,96.0,96.0,96.0,88.0,84.0,100.0,100.0,76.0
1,건강점수,강동구,100,1,75,3,100.0,100.0,46.0,92.0,92.0,96.0,4.0,70.0
2,건강점수,강북구,60,9,50,11,44.0,76.0,29.0,96.0,40.0,24.0,44.0,52.0
3,건강점수,강서구,4,25,32,23,4.0,4.0,21.0,12.0,20.0,64.0,92.0,44.0
4,건강점수,관악구,8,24,36,21,8.0,8.0,38.0,4.0,56.0,68.0,96.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,관리점수,용산구,63,7,61,8,72.0,36.0,83.0,28.0,68.0,92.0,16.0,96.0
71,관리점수,은평구,49,13,39,18,64.0,44.0,NaN,84.0,60.0,4.0,32.0,24.0
72,관리점수,종로구,37,19,63,7,52.0,92.0,79.0,20.0,48.0,44.0,76.0,100.0
73,관리점수,중구,80,3,74,4,92.0,88.0,58.0,68.0,100.0,72.0,24.0,92.0


In [ ]:
# 데이터 내보내기
all_data.drop('할당량', axis=1, inplace=True)
all_data.to_csv(path_or_buf='./dog_map_df.csv',encoding='utf-8-sig',index=False)
all_score.to_csv(path_or_buf='./dog_map_score.csv',encoding='utf-8-sig',index=False)